In [1]:
from sympy import *
from sympy.simplify.fu import sincos_to_sum

In [2]:
x, y, z, eps, k, S = symbols('x y z varepsilon k S')
xi = Matrix([x, y, z])

In [3]:
zeta_d, zeta_1, zeta_2 = symbols('zeta_d zeta_1 zeta_2')
alpha = symbols('alpha')

In [4]:
a0, a1, b0, b1, c0, c1 = symbols('a_0 a_1 b_0 b_1 c_0 c_1')
A0, A1, B0, B1, C0, C1 = symbols('A_0 A_1 B_0 B_1 C_0 C_1')

In [5]:
subs_dict = {S : 0.6751, k : 0.0810, eps : 0.01,
             zeta_d : 1.0050, zeta_1 : 1.0050, zeta_2 : -1.7971,
             alpha : 8.0,
             a0 : 3.6008289, a1 : -2.337e-4,
             b0 : 0.0, b1 : 5.403e-2,
             c0 : -1.8010167, c1 : 7.191e-4,
             A0 : 0.0, A1 : 0.1385,
             B0 : 8.004373, B1 : -4.235e-3,
             C0 : 0.0, C1 : -0.4263}

## Generate $Q$-tensor

In [6]:
Q_mat = S * Matrix([[Rational(2, 3), eps*sin(k*x), 0],
                    [eps*sin(k*x), -Rational(1, 3), 0],
                    [0, 0, -Rational(1, 3)]])

Q_mat

Matrix([
[                2*S/3, S*varepsilon*sin(k*x),    0],
[S*varepsilon*sin(k*x),                  -S/3,    0],
[                    0,                     0, -S/3]])

## Generate elastic stress tensor and corresponding force

In [7]:
sigma_e = zeros(3)
for i in range(3):
    for j in range(3):
        sigma_e[i, j] = -sum([Q_mat[k, l].diff(xi[i]) * Q_mat[k, l].diff(xi[j])
                              for k in range(3)
                              for l in range(3)])

In [8]:
sigma_e

Matrix([
[-2*S**2*k**2*varepsilon**2*cos(k*x)**2, 0, 0],
[                                     0, 0, 0],
[                                     0, 0, 0]])

In [9]:
f_e = zeros(3, 1)
for i in range(3):
    f_e[i, 0] = sum([sigma_e[k, i].diff(xi[k])
                     for k in range(3)])
f_e = trigsimp(f_e)
f_e

Matrix([
[2*S**2*k**3*varepsilon**2*sin(2*k*x)],
[                                   0],
[                                   0]])

In [10]:
f_ex = f_e[0]
f_ex_1 = f_ex.coeff(sin(2*k*x))
f_ex_1

2*S**2*k**3*varepsilon**2

In [11]:
print("p_2 is given by:", (f_ex_1 / (-2*k)).subs(subs_dict))

p_2 is given by: -2.99024142561000e-7


### Check stress tensor amplitudes

In [12]:
sigma_e_1 = expand(sincos_to_sum(sigma_e)[0, 0])
sigma_e_1 = collect(sigma_e_1, cos(2*k*x))
sigma_e_1

-S**2*k**2*varepsilon**2*cos(2*k*x) - S**2*k**2*varepsilon**2

In [13]:
sigma_e_amplitude = sigma_e_1.coeff(cos(2*k*x))
sigma_e_amplitude

-S**2*k**2*varepsilon**2

In [14]:
sigma_e_offset = sigma_e_1.args[0]
sigma_e_offset

-S**2*k**2*varepsilon**2

In [15]:
print("Elastic stress tensor amplitude is:", sigma_e_amplitude.subs(subs_dict))
print("Elastic stress tensor offset is:", sigma_e_offset.subs(subs_dict))

Elastic stress tensor amplitude is: -2.99024142561000e-7
Elastic stress tensor offset is: -2.99024142561000e-7


## Generate $\mu_2$ viscous stress tensor and corresponding force

In [16]:
Lambda1 = Function("\Lambda_1")(x)
Lambda2 = Function("\Lambda_2")(x)
Lambda4 = Function("\Lambda_4")(x)
Lambda_mat = Matrix([[Lambda1, Lambda2, 0],
                     [Lambda2, Lambda4, 0],
                     [0, 0, -(Lambda1 + Lambda4)]])
Lambda_mat

Matrix([
[\Lambda_1(x), \Lambda_2(x),                            0],
[\Lambda_2(x), \Lambda_4(x),                            0],
[           0,            0, -\Lambda_1(x) - \Lambda_4(x)]])

In [17]:
dLambda1_dQ2 = symbols('\\frac{\partial\Lambda_1}{\partial\ Q_2}')
dLambda2_dQ2 = symbols('\\frac{\partial\Lambda_2}{\partial\ Q_2}')
dLambda4_dQ2 = symbols('\\frac{\partial\Lambda_4}{\partial\ Q_2}')

In [18]:
lambda_subs_dict = {Lambda1 : a0 + a1 * cos(2*k*x),
                    Lambda2 : b0 + b1 * sin(k*x),
                    Lambda4 : c0 + c1 * cos(2*k*x),
                    dLambda1_dQ2 : A0 + A1 * sin(k*x),
                    dLambda2_dQ2 : B0 + B1 * cos(2*k*x),
                    dLambda4_dQ2 : C0 + C1 * sin(k*x)}

In [19]:
lap_Q_mat = zeros(3)
for i in range(3):
    for j in range(3):
        lap_Q_mat[i, j] = sum([Q_mat[i, j].diff(xi[k]).diff(xi[k])
                              for k in range(3)])
        
h = alpha*Q_mat - Lambda_mat + lap_Q_mat
h

Matrix([
[                                             2*S*alpha/3 - \Lambda_1(x), S*alpha*varepsilon*sin(k*x) - S*k**2*varepsilon*sin(k*x) - \Lambda_2(x),                                        0],
[S*alpha*varepsilon*sin(k*x) - S*k**2*varepsilon*sin(k*x) - \Lambda_2(x),                                               -S*alpha/3 - \Lambda_4(x),                                        0],
[                                                                      0,                                                                       0, -S*alpha/3 + \Lambda_1(x) + \Lambda_4(x)]])

In [20]:
f_h = zeros(3, 1)
for i in range(3):
    f_h[i, 0] = sum([h[k, i].diff(xi[k])
                     for k in range(3)])

f_h

Matrix([
[                                                            -Derivative(\Lambda_1(x), x)],
[S*alpha*k*varepsilon*cos(k*x) - S*k**3*varepsilon*cos(k*x) - Derivative(\Lambda_2(x), x)],
[                                                                                       0]])

In [21]:
Q_vec = zeros(5, 1)
Q_vec[0, 0] = Q_mat[0, 0]
Q_vec[1, 0] = Q_mat[0, 1]
Q_vec[2, 0] = Q_mat[0, 2]
Q_vec[3, 0] = Q_mat[1, 1]
Q_vec[4, 0] = Q_mat[1, 2]

Q_vec

Matrix([
[                2*S/3],
[S*varepsilon*sin(k*x)],
[                    0],
[                 -S/3],
[                    0]])

In [22]:
Q_vec.diff(x)

Matrix([
[                      0],
[S*k*varepsilon*cos(k*x)],
[                      0],
[                      0],
[                      0]])

In [23]:
d_lambda_subs_dict = {Lambda1.diff(x) : Q_vec[1].diff(x) * dLambda1_dQ2,
                      Lambda2.diff(x) : Q_vec[1].diff(x) * dLambda2_dQ2,
                      Lambda4.diff(x) : Q_vec[1].diff(x) * dLambda4_dQ2}

In [24]:
f_h = f_h.subs(d_lambda_subs_dict)
f_h = trigsimp(f_h)
f_h

Matrix([
[                 -S*\frac{\partial\Lambda_1}{\partial Q_2}*k*varepsilon*cos(k*x)],
[S*k*varepsilon*(-\frac{\partial\Lambda_2}{\partial Q_2} + alpha - k**2)*cos(k*x)],
[                                                                               0]])

In [25]:
f_h = f_h.subs(lambda_subs_dict)
f_h = sincos_to_sum(expand(f_h))
f_h

Matrix([
[                                                                           -A_0*S*k*varepsilon*cos(k*x) - A_1*S*k*varepsilon*sin(2*k*x)/2],
[-B_0*S*k*varepsilon*cos(k*x) + B_1*S*k*varepsilon*(-cos(k*x) - cos(3*k*x))/2 + S*alpha*k*varepsilon*cos(k*x) - S*k**3*varepsilon*cos(k*x)],
[                                                                                                                                        0]])

In [26]:
f_hx = collect(expand(f_h[0]), sin(k*x))
f_hy = collect(expand(f_h[1]), (cos(k*x), cos(3*k*x)))

In [27]:
f_hx

-A_0*S*k*varepsilon*cos(k*x) - A_1*S*k*varepsilon*sin(2*k*x)/2

In [28]:
f_hy

-B_1*S*k*varepsilon*cos(3*k*x)/2 + (-B_0*S*k*varepsilon - B_1*S*k*varepsilon/2 + S*alpha*k*varepsilon - S*k**3*varepsilon)*cos(k*x)

In [29]:
f_hx_1 = f_hx.coeff(cos(k*x))
f_hx_2 = f_hx.coeff(sin(2*k*x))

f_hy_1 = f_hy.coeff(cos(k*x))
f_hy_3 = f_hy.coeff(cos(3*k*x))

In [30]:
f_hx_1

-A_0*S*k*varepsilon

In [31]:
f_hx_2

-A_1*S*k*varepsilon/2

In [32]:
f_hy_1

-B_0*S*k*varepsilon - B_1*S*k*varepsilon/2 + S*alpha*k*varepsilon - S*k**3*varepsilon

In [33]:
f_hy_3

-B_1*S*k*varepsilon/2

In [34]:
print("p_2 is given by:", (zeta_2 * f_hx_2 / (-2*k)).subs(subs_dict))
print("v_1 is given by:", (zeta_2 * f_hy_1 / k**2).subs(subs_dict))
print("v_3 is given by:", (zeta_2 * f_hy_3 / (9 * k**2)).subs(subs_dict))

p_2 is given by: -0.000420078190212500
v_1 is given by: 0.00132053995240282
v_3 is given by: -3.52400278419067e-5


### Check stress tensor amplitudes

In [35]:
h_exp = h.subs(lambda_subs_dict)
for i in range(3):
    for j in range(3):
        h_exp[i, j] = collect(h_exp[i, j], (sin(k*x), cos(2*k*x), 1))
h_exp

Matrix([
[                            2*S*alpha/3 - a_0 - a_1*cos(2*k*x), -b_0 + (S*alpha*varepsilon - S*k**2*varepsilon - b_1)*sin(k*x),                                               0],
[-b_0 + (S*alpha*varepsilon - S*k**2*varepsilon - b_1)*sin(k*x),                              -S*alpha/3 - c_0 - c_1*cos(2*k*x),                                               0],
[                                                             0,                                                              0, -S*alpha/3 + a_0 + c_0 + (a_1 + c_1)*cos(2*k*x)]])

In [36]:
print(latex(h_exp))

\left[\begin{matrix}\frac{2 S \alpha}{3} - a_{0} - a_{1} \cos{\left(2 k x \right)} & - b_{0} + \left(S \alpha \varepsilon - S k^{2} \varepsilon - b_{1}\right) \sin{\left(k x \right)} & 0\\- b_{0} + \left(S \alpha \varepsilon - S k^{2} \varepsilon - b_{1}\right) \sin{\left(k x \right)} & - \frac{S \alpha}{3} - c_{0} - c_{1} \cos{\left(2 k x \right)} & 0\\0 & 0 & - \frac{S \alpha}{3} + a_{0} + c_{0} + \left(a_{1} + c_{1}\right) \cos{\left(2 k x \right)}\end{matrix}\right]


In [37]:
h_exp.subs(subs_dict)

Matrix([
[0.0002337*cos(0.162*x) - 0.000295566666666858,             -6.62933109999986e-5*sin(0.081*x),                                             0],
[            -6.62933109999986e-5*sin(0.081*x), 0.000750033333333233 - 0.0007191*cos(0.162*x),                                             0],
[                                            0,                                             0, 0.0004854*cos(0.162*x) - 0.000454466666666375]])

## Generate $\mu_1$ stress tensor and corresponding force

In [38]:
sigma_mu1 = h * Q_mat - Q_mat * h
for i in range(3):
    for j in range(3):
        sigma_mu1[i, j] = simplify(sigma_mu1[i, j])
sigma_mu1

Matrix([
[                                                                                                                   0, S*(S*k**2*varepsilon*sin(k*x) - varepsilon*\Lambda_1(x)*sin(k*x) + varepsilon*\Lambda_4(x)*sin(k*x) + \Lambda_2(x)), 0],
[S*(-S*k**2*varepsilon*sin(k*x) + varepsilon*\Lambda_1(x)*sin(k*x) - varepsilon*\Lambda_4(x)*sin(k*x) - \Lambda_2(x)),                                                                                                                   0, 0],
[                                                                                                                   0,                                                                                                                   0, 0]])

In [39]:
f_mu1 = zeros(3, 1)
for i in range(3):
    f_mu1[i, 0] = sum([sigma_mu1[k, i].diff(xi[k])
                       for k in range(3)])

f_mu1 = simplify(f_mu1)
f_mu1

Matrix([
[                                                                                                                                                                                                                                         0],
[S*(S*k**3*varepsilon*cos(k*x) - k*varepsilon*\Lambda_1(x)*cos(k*x) + k*varepsilon*\Lambda_4(x)*cos(k*x) - varepsilon*sin(k*x)*Derivative(\Lambda_1(x), x) + varepsilon*sin(k*x)*Derivative(\Lambda_4(x), x) + Derivative(\Lambda_2(x), x))],
[                                                                                                                                                                                                                                         0]])

In [40]:
f_mu1 = f_mu1.subs(d_lambda_subs_dict)
f_mu1 = sincos_to_sum(f_mu1)
f_mu1

Matrix([
[                                                                                                                                                                                                                                                                                                                                     0],
[-S**2*\frac{\partial\Lambda_1}{\partial Q_2}*k*varepsilon**2*sin(2*k*x)/2 + S**2*\frac{\partial\Lambda_2}{\partial Q_2}*k*varepsilon*cos(k*x) + S**2*\frac{\partial\Lambda_4}{\partial Q_2}*k*varepsilon**2*sin(2*k*x)/2 + S**2*k**3*varepsilon*cos(k*x) - S*k*varepsilon*\Lambda_1(x)*cos(k*x) + S*k*varepsilon*\Lambda_4(x)*cos(k*x)],
[                                                                                                                                                                                                                                                                                                                                     0]])

In [41]:
f_mu1 = f_mu1.subs(lambda_subs_dict)
f_mu1 = expand(simplify(sincos_to_sum(f_mu1)))
f_mu1

Matrix([
[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  0],
[-A_0*S**2*k*varepsilon**2*sin(2*k*x)/2 - A_1*S**2*k*varepsilon**2*cos(k*x)/4 + A_1*S**2*k*varepsilon**2*cos(3*k*x)/4 + B_0*S**2*k*varepsilon*cos(k*x) + B_1*S**2*k*varepsilon*cos(k*x)/2 + B_1*S**2*k*varepsilon*cos(3*k*x)/2 + C_0*S**2*k*varepsilon**2*sin(2*k*x)/2 + C_1*S**2*k*varepsilon**2*cos(k*x)/4 - C_1*S**2*k*varepsilon**2*cos(3*k*x)/4 + S**2*k**3*varepsilon*cos(k*x) - S*a_0*k*varepsilon*cos(k*x) - S*a_1*k*varepsilon*

In [42]:
f_mu1_expr = collect(f_mu1[1], (cos(k*x), sin(2*k*x), cos(3*k*x)))
f_mu1_expr

(-A_0*S**2*k*varepsilon**2/2 + C_0*S**2*k*varepsilon**2/2)*sin(2*k*x) + (A_1*S**2*k*varepsilon**2/4 + B_1*S**2*k*varepsilon/2 - C_1*S**2*k*varepsilon**2/4 - S*a_1*k*varepsilon/2 + S*c_1*k*varepsilon/2)*cos(3*k*x) + (-A_1*S**2*k*varepsilon**2/4 + B_0*S**2*k*varepsilon + B_1*S**2*k*varepsilon/2 + C_1*S**2*k*varepsilon**2/4 + S**2*k**3*varepsilon - S*a_0*k*varepsilon - S*a_1*k*varepsilon/2 + S*c_0*k*varepsilon + S*c_1*k*varepsilon/2)*cos(k*x)

In [43]:
c_mu1_1 = simplify(f_mu1_expr.coeff(cos(k*x)))
c_mu1_1

S*k*varepsilon*(-A_1*S*varepsilon + 4*B_0*S + 2*B_1*S + C_1*S*varepsilon + 4*S*k**2 - 4*a_0 - 2*a_1 + 4*c_0 + 2*c_1)/4

In [44]:
print(latex(c_mu1_1))

\frac{S k \varepsilon \left(- A_{1} S \varepsilon + 4 B_{0} S + 2 B_{1} S + C_{1} S \varepsilon + 4 S k^{2} - 4 a_{0} - 2 a_{1} + 4 c_{0} + 2 c_{1}\right)}{4}


In [45]:
c_mu1_2 = simplify(f_mu1_expr.coeff(sin(2*k*x)))
c_mu1_2

S**2*k*varepsilon**2*(-A_0 + C_0)/2

In [46]:
c_mu1_3 = simplify(f_mu1_expr.coeff(cos(3*k*x)))
c_mu1_3

S*k*varepsilon*(A_1*S*varepsilon + 2*B_1*S - C_1*S*varepsilon - 2*a_1 + 2*c_1)/4

In [47]:
print(latex(c_mu1_3))

\frac{S k \varepsilon \left(A_{1} S \varepsilon + 2 B_{1} S - C_{1} S \varepsilon - 2 a_{1} + 2 c_{1}\right)}{4}


In [48]:
print("v_1 is given by:", (c_mu1_1 * zeta_1 / k**2).subs(subs_dict))
print("v_3 is given by:", (c_mu1_3 * zeta_1 / (9 * k**2)).subs(subs_dict))
print("c_mu1_2 is given by:", c_mu1_2.subs(subs_dict))

v_1 is given by: 0.000371032112890673
v_3 is given by: 1.08844594958730e-9
c_mu1_2 is given by: 0


### Check $\mu_1$ stress tensor values

In [49]:
sigma_mu1_expr = sigma_mu1.subs(lambda_subs_dict)
for i in range(3):
    for j in range(3):
        sigma_mu1_expr[i, j] = sincos_to_sum(sigma_mu1_expr[i, j])
        sigma_mu1_expr[i, j] = collect(expand(sigma_mu1_expr[i, j]), (sin(k*x), sin(3*k*x)))
sigma_mu1_expr

Matrix([
[                                                                                                                                                                                       0, S*b_0 + (-S*a_1*varepsilon/2 + S*c_1*varepsilon/2)*sin(3*k*x) + (S**2*k**2*varepsilon - S*a_0*varepsilon + S*a_1*varepsilon/2 + S*b_1 + S*c_0*varepsilon - S*c_1*varepsilon/2)*sin(k*x), 0],
[-S*b_0 + (S*a_1*varepsilon/2 - S*c_1*varepsilon/2)*sin(3*k*x) + (-S**2*k**2*varepsilon + S*a_0*varepsilon - S*a_1*varepsilon/2 - S*b_1 - S*c_0*varepsilon + S*c_1*varepsilon/2)*sin(k*x),                                                                                                                                                                                       0, 0],
[                                                                                                                                                                                       0,                                                     

In [50]:
print(latex(sigma_mu1_expr[0, 1]))

S b_{0} + \left(- \frac{S a_{1} \varepsilon}{2} + \frac{S c_{1} \varepsilon}{2}\right) \sin{\left(3 k x \right)} + \left(S^{2} k^{2} \varepsilon - S a_{0} \varepsilon + \frac{S a_{1} \varepsilon}{2} + S b_{1} + S c_{0} \varepsilon - \frac{S c_{1} \varepsilon}{2}\right) \sin{\left(k x \right)}


In [51]:
sigma_mu1_0 = sigma_mu1_expr[0, 1].args[0]
sigma_mu1_1 = sigma_mu1_expr[0, 1].coeff(sin(k*x))
sigma_mu1_3 = sigma_mu1_expr[0, 1].coeff(sin(3*k*x))

In [52]:
print("Constant is:", sigma_mu1_0.subs(subs_dict))
print("sin(kx) is:", sigma_mu1_1.subs(subs_dict))
print("sin(3kx) is:", sigma_mu1_3.subs(subs_dict))

Constant is: 0
sin(kx) is: 3.44795922561092e-5
sin(3kx) is: 3.21617640000000e-6


In [53]:
sigma_mu1_3

-S*a_1*varepsilon/2 + S*c_1*varepsilon/2

In [54]:
sigma_mu1_1

S**2*k**2*varepsilon - S*a_0*varepsilon + S*a_1*varepsilon/2 + S*b_1 + S*c_0*varepsilon - S*c_1*varepsilon/2